# Exploration of Dimension Reduction
<span style="font-weight:bold; font-size:17pt; color:#666666;">XGBoost for feature selection</span>
<hr>

This notebook is for EDA, feature extraction, engineering and the subsequent evaluation of dimension reduction techniques.

It assumes the data (**features.csv**, the full feature set) is in a sub-directory of the **/data** folder. I've already added entries to the _.gitignore_ file so that they won't be committed to the repository. Note that this file should be updated for new versions of the data.

See the [data readme in the Gitbug repository](https://github.com/BrianDavisMath/FDA-COVID19/tree/master/data) for more details.

<hr>

In [1]:
%pylab inline
%autosave 25

import pandas as pd

Populating the interactive namespace from numpy and matplotlib


Autosaving every 25 seconds


## Data location

Change this when you get a new data set.

In [2]:
data_loc = '../data/FDA-COVID19_files_v0.5/'

## Load the data from features.csv
<hr>

In [ ]:
df_features = pd.read_csv(data_loc+'features.csv')
print('rows: {:,}, columns: {:,}'.format(len(df_features), len(df_features.columns)))

## Experiment 1

XGBoost for features extraction.

* decompose features into dataframes with and without pid, cid and activity data etc.
* take a uniform random sample of 10,000 points 
* split into train and test sets <br>
* scale the training set <br>
* scale the test set with the same scaler <br>
* train the model
* tweak the parameters
* retrain, rinse and repeat

In [5]:
from sklearn.preprocessing import MinMaxScaler

# store for later
df_y = df_features['activity']
df_ids = df_features[['pid', 'cid']]

# Select only numeric columns and drop pid, cid and activity.
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64'] # pid dropped because it's a string
number_cols = df_features.select_dtypes(include=numerics)
number_cols = [col for col in number_cols]
number_cols.remove('activity')
number_cols.remove('cid')

print('Number of rows: {:,}'.format(len(df_features)))
print('Number of columns: {:,}'.format(len(number_cols)))

df_data = df_features[number_cols]

Number of rows: 158,806
Number of columns: 4,518


In [ ]:
df_sample = df_data.sample(n=10000, axis=0)
print('Number of sample rows: {:,}'.format(len(df_sample)))
print('Number of samepl columns: {:,}'.format(len(df_sample)))